In [3]:
import pandas as pd
import matplotlib as plt
%matplotlib inline

In [4]:
dataset = pd.read_csv('binary.csv', sep=',', header=0)

In [5]:
dataset.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [6]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [8]:
data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
209,0,-0.066657,0.289305,0,1,0,0
280,0,0.625884,1.445476,0,1,0,0
33,1,1.837832,1.603135,0,0,1,0
210,0,1.318426,-0.131120,0,0,0,1
93,0,-0.066657,-1.208461,0,1,0,0


In [27]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.

# Derivativa da sigmoid
def sigmoid_prime(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 2000
learnrate = 0.25

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output
        h = np.dot(weights, x)

        # TODO: Calculate the output
        output = sigmoid(h)

        # TODO: Calculate the error
        error = y - output
        
        # Gradient
        output_grad = sigmoid_prime(h)

        # TODO: Calculate the error term
        error_term = error * output_grad

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.247491331957
Train loss:  0.21956346761
Train loss:  0.231404638162   WARNING - Loss Increasing
Train loss:  0.218724721265
Train loss:  0.235414817386   WARNING - Loss Increasing
Train loss:  0.220299921012
Train loss:  0.211076425977
Train loss:  0.260166775688   WARNING - Loss Increasing
Train loss:  0.210830487712
Train loss:  0.239961247518   WARNING - Loss Increasing
Prediction accuracy: 0.800
